In [1]:
import cv2 
import numpy as np
from keras.models import load_model
import time
def Detect_Faces(image, model):
    return model.detectMultiScale(image, 1.3, 5)  # Default Values

def Detect_Faces(image, model):
    return model.detectMultiScale(image, 1.3, 5)  # Default Values

def Draw_Bounding_BOX(Face_points, image, color_box):
    overlay_face = image.copy()
    overlay_top = image.copy()
    alpha_face=0.8
    alpha_tope=0.5
    shift = 2
    x_point, y_point, width, height = Face_points
    x_offset=int(width/20.)
    y_offset=int(height/20.)
    # print(Face_points)
    cv2.rectangle(overlay_face, (x_point + x_offset, y_point - y_offset), (x_point + width - x_offset, y_point + height ), color_box, shift)
    cv2.rectangle(overlay_top, (x_point + x_offset - shift // 2, y_point - height//20), (x_point + width - x_offset + shift // 2, y_point - height//5), (255,128,0), -1)
    cv2.addWeighted(overlay_face, alpha_face, image, 1 - alpha_face, 0, image)
    cv2.addWeighted(overlay_top, alpha_tope, image, 1 - alpha_tope, 0, image)
    
def put_emoji(image,Face_points,Emoji_images,emotion_num):
    size_mult=1
    x_point, y_point, width, height = Face_points
    height_rec = 0.15*height
    offset = int(height_rec*0.1)
    
    emotion_size = int(height_rec*size_mult)
    
    x2 = int(x_point + width - offset*3)
    x1 = int(x2 - emotion_size )
    
    y1 = int(y_point - offset*4)
    y2 = int(y1- emotion_size)

    emoji_img = cv2.resize(Emoji_images[emotion_num],(int(emotion_size),int(emotion_size)))
    mask =np.array((emoji_img==255), np.uint8)

    image[y2:y1,x1:x2] =  image[y2:y1,x1:x2]*mask + emoji_img*(1-mask)  # dont forget in lost [smaller : larger] no the reverse
    return image
    
    
    
def ROI_Face_Frame(image,Face_points):
    x_point, y_point, width, height = Face_points
    x_offset=int(width/20.)
    y_offset=int(height/20.)
    x1=x_point + x_offset
    x2=x_point + width - x_offset
    y1=y_point - y_offset
    y2=y_point + height 
    face = image[y1:y2,x1:x2]
    return face



Using TensorFlow backend.


In [2]:
#load models 
model1 = 'weights.51-1.19.hdf5'
model2 ='weights.37-0.91.hdf5'
model3 ='model_weights.h5'
model4 ='weights.08-1.19.hdf5'
model5 = 'mini_XCEPTION_KDEF.hdf5'
model6 = 'tiny_XCEPTION_KDEF.hdf5'
model7 = 'simple_CNN.985-0.66.hdf5'
model8 = 'simple_CNN.530-0.65.hdf5'
model9 = 'fer2013_mini_XCEPTION.102-0.66.hdf5'
model10 = 'weights.51-0.313-conv-32-nodes-0-dense-1568549063.hdf5'

model11 = 'weights.37-0.29-4-conv-256-nodes-0-dense-1568581634.hdf5'
model12 = 'weights.46-0.28-4-conv-128-nodes-1-dense-1568590843.hdf5'
model13 = 'weights.13-0.29-5-conv-64-nodes-0-dense-1568685072-Trial2.hdf5'
emotion_classifier = load_model(model13, compile=False)
face_detection_model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")




Instructions for updating:
Colocations handled automatically by placer.


In [13]:
from mtcnn.mtcnn import MTCNN
detector = MTCNN()

cv2.namedWindow('Live')
video_capture = cv2.VideoCapture(0)
EMOTIONS_LIST = ['angry', 'happy', 'sad', 'surprise', 'neutral']
Emoji_images = [cv2.imread('Emojis\{}.png'.format(no)) for no in range(1,8)]
size = (int(video_capture.read()[1].shape[1]*1.8),int(video_capture.read()[1].shape[0]*1.8))
Markov = cv2.resize(cv2.imread('markov.png') , size )
cv2.imshow('Live',Markov)
cv2.waitKey(5000)
thanks = cv2.resize(cv2.imread('thanks.png') , size )
cv2.imshow('Live',thanks)
cv2.waitKey(3000)


while True:
    bgr_frame = video_capture.read()[1]
    result =(detector.detect_faces(bgr_frame))
    if result != []:
        print(result)
        bounding_box = result[0]['box']
        cv2.rectangle(bgr_frame,
                      (bounding_box[0], bounding_box[1]),
                      (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                      (0,155,255),
                      2)
    #bgr_frame=cv2.resize(bgr_frame,(int(bgr_frame.shape[1]*1.5),int(bgr_frame.shape[0]*1.5)))
    gray_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2GRAY)
    #print(bgr_frame.shape)
    #gray_frame= cv2.add(gray_frame,np.array([50.0]))
    face_points = Detect_Faces(gray_frame,face_detection_model)
    for one_face_points in face_points:
        if one_face_points[2]*one_face_points[3] > 10000:
           # print(one_face_points[2]*one_face_points[3] )
            Draw_Bounding_BOX(one_face_points,bgr_frame,[0,0,0])
            ROI_Face=ROI_Face_Frame(gray_frame,one_face_points)
            #cv2.imshow('face', ROI_Face)
            ROI_Face_Predict = cv2.resize(ROI_Face.copy(),(48,48))/255.
            ROI_Face_Predict=ROI_Face_Predict[np.newaxis, :, :, np.newaxis]
           # print(ROI_Face_Predict.shape)
            custom = emotion_classifier.predict(ROI_Face_Predict)*100.
            emotion_num = np.argmax(custom)
            put_emoji(bgr_frame, one_face_points, Emoji_images, emotion_num)
            emotion = EMOTIONS_LIST[emotion_num] + " " + str(int(np.max(custom))) + " %"
            cv2.putText(bgr_frame, emotion, (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
       # print(custom)
    
    bgr_frame=cv2.resize(bgr_frame,(int(bgr_frame.shape[1]*1.8),int(bgr_frame.shape[0]*1.8)))
    cv2.imshow('Live', bgr_frame)
    #cv2.imshow('grey', gray_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()


[{'box': [265, 228, 115, 154], 'confidence': 0.9975970387458801, 'keypoints': {'left_eye': (296, 289), 'right_eye': (351, 290), 'nose': (323, 323), 'mouth_left': (301, 349), 'mouth_right': (344, 350)}}]
[{'box': [262, 226, 118, 159], 'confidence': 0.9974315762519836, 'keypoints': {'left_eye': (294, 288), 'right_eye': (351, 289), 'nose': (324, 324), 'mouth_left': (300, 349), 'mouth_right': (345, 349)}}]
[{'box': [263, 215, 126, 169], 'confidence': 0.9995100498199463, 'keypoints': {'left_eye': (299, 288), 'right_eye': (355, 285), 'nose': (329, 328), 'mouth_left': (301, 350), 'mouth_right': (352, 349)}}]
[{'box': [263, 215, 123, 171], 'confidence': 0.9995225667953491, 'keypoints': {'left_eye': (302, 289), 'right_eye': (359, 286), 'nose': (332, 329), 'mouth_left': (302, 352), 'mouth_right': (354, 351)}}]
[{'box': [264, 218, 127, 172], 'confidence': 0.998870313167572, 'keypoints': {'left_eye': (302, 290), 'right_eye': (360, 286), 'nose': (333, 331), 'mouth_left': (304, 353), 'mouth_right': 

[{'box': [226, 221, 109, 153], 'confidence': 0.9926693439483643, 'keypoints': {'left_eye': (243, 282), 'right_eye': (289, 279), 'nose': (251, 309), 'mouth_left': (244, 340), 'mouth_right': (286, 340)}}]
[{'box': [227, 226, 105, 142], 'confidence': 0.9943286776542664, 'keypoints': {'left_eye': (244, 282), 'right_eye': (291, 280), 'nose': (255, 309), 'mouth_left': (247, 340), 'mouth_right': (287, 340)}}]
[{'box': [227, 223, 106, 146], 'confidence': 0.9950939416885376, 'keypoints': {'left_eye': (245, 280), 'right_eye': (294, 278), 'nose': (257, 308), 'mouth_left': (247, 340), 'mouth_right': (290, 341)}}]
[{'box': [242, 221, 115, 143], 'confidence': 0.9967928528785706, 'keypoints': {'left_eye': (266, 275), 'right_eye': (320, 276), 'nose': (282, 301), 'mouth_left': (265, 333), 'mouth_right': (312, 334)}}]
[{'box': [280, 242, 111, 130], 'confidence': 0.8666741847991943, 'keypoints': {'left_eye': (322, 275), 'right_eye': (373, 289), 'nose': (343, 307), 'mouth_left': (310, 333), 'mouth_right':

[{'box': [227, 217, 115, 156], 'confidence': 0.997444748878479, 'keypoints': {'left_eye': (255, 275), 'right_eye': (313, 275), 'nose': (283, 304), 'mouth_left': (261, 338), 'mouth_right': (308, 338)}}]
[{'box': [230, 219, 113, 157], 'confidence': 0.9908864498138428, 'keypoints': {'left_eye': (255, 277), 'right_eye': (312, 277), 'nose': (282, 308), 'mouth_left': (260, 339), 'mouth_right': (307, 340)}}]
[{'box': [233, 217, 116, 160], 'confidence': 0.9910796284675598, 'keypoints': {'left_eye': (260, 276), 'right_eye': (316, 278), 'nose': (285, 311), 'mouth_left': (263, 339), 'mouth_right': (312, 342)}}]
[{'box': [238, 219, 111, 153], 'confidence': 0.9932563304901123, 'keypoints': {'left_eye': (264, 277), 'right_eye': (319, 278), 'nose': (289, 309), 'mouth_left': (267, 337), 'mouth_right': (314, 340)}}]
[{'box': [237, 219, 113, 153], 'confidence': 0.9976385831832886, 'keypoints': {'left_eye': (263, 277), 'right_eye': (318, 278), 'nose': (289, 309), 'mouth_left': (268, 336), 'mouth_right': 

[{'box': [231, 216, 118, 153], 'confidence': 0.9990731477737427, 'keypoints': {'left_eye': (260, 276), 'right_eye': (319, 274), 'nose': (290, 304), 'mouth_left': (268, 337), 'mouth_right': (315, 337)}}]
[{'box': [232, 217, 117, 152], 'confidence': 0.999206006526947, 'keypoints': {'left_eye': (260, 274), 'right_eye': (319, 275), 'nose': (289, 304), 'mouth_left': (266, 335), 'mouth_right': (314, 337)}}]
[{'box': [234, 217, 118, 156], 'confidence': 0.9985719919204712, 'keypoints': {'left_eye': (263, 276), 'right_eye': (320, 275), 'nose': (290, 305), 'mouth_left': (268, 338), 'mouth_right': (315, 337)}}]
[{'box': [237, 215, 115, 156], 'confidence': 0.9957388639450073, 'keypoints': {'left_eye': (265, 274), 'right_eye': (321, 277), 'nose': (292, 307), 'mouth_left': (268, 334), 'mouth_right': (315, 337)}}]
[{'box': [237, 217, 116, 156], 'confidence': 0.9959681034088135, 'keypoints': {'left_eye': (263, 275), 'right_eye': (320, 276), 'nose': (289, 308), 'mouth_left': (268, 338), 'mouth_right': 

[{'box': [233, 220, 118, 153], 'confidence': 0.9987578392028809, 'keypoints': {'left_eye': (258, 278), 'right_eye': (317, 277), 'nose': (288, 306), 'mouth_left': (265, 339), 'mouth_right': (314, 338)}}]
[{'box': [234, 219, 118, 153], 'confidence': 0.9984733462333679, 'keypoints': {'left_eye': (260, 277), 'right_eye': (319, 276), 'nose': (288, 306), 'mouth_left': (267, 338), 'mouth_right': (315, 337)}}]
[{'box': [237, 217, 113, 150], 'confidence': 0.9980676770210266, 'keypoints': {'left_eye': (265, 274), 'right_eye': (319, 275), 'nose': (291, 304), 'mouth_left': (270, 332), 'mouth_right': (314, 333)}}]
[{'box': [235, 207, 109, 136], 'confidence': 0.9867238402366638, 'keypoints': {'left_eye': (261, 258), 'right_eye': (316, 256), 'nose': (288, 277), 'mouth_left': (269, 312), 'mouth_right': (313, 311)}}]
[{'box': [238, 203, 108, 133], 'confidence': 0.9945178627967834, 'keypoints': {'left_eye': (262, 251), 'right_eye': (315, 250), 'nose': (286, 268), 'mouth_left': (268, 303), 'mouth_right':